In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/prediction-with-goa-uniprot-all/ids_test.txt
/kaggle/input/prediction-with-goa-uniprot-all/submission.tsv
/kaggle/input/prediction-with-goa-uniprot-all/__results__.html
/kaggle/input/prediction-with-goa-uniprot-all/__notebook__.ipynb
/kaggle/input/prediction-with-goa-uniprot-all/__output__.json
/kaggle/input/prediction-with-goa-uniprot-all/custom.css
/kaggle/input/k/daovanda2405/cafa-6-protein-function-starter-eda-model/submission.tsv
/kaggle/input/k/daovanda2405/cafa-6-protein-function-starter-eda-model/__results__.html
/kaggle/input/k/daovanda2405/cafa-6-protein-function-starter-eda-model/__notebook__.ipynb
/kaggle/input/k/daovanda2405/cafa-6-protein-function-starter-eda-model/__output__.json
/kaggle/input/k/daovanda2405/cafa-6-protein-function-starter-eda-model/custom.css
/kaggle/input/k/daovanda2405/cafa-6-protein-function-starter-eda-model/__results___files/__results___1_3.png
/kaggle/input/k/daovanda2405/cafa-6-protein-function-starter-eda-model/__results___files/__

In [2]:
import numpy as np
import pandas as pd
import os
import time
from tqdm.auto import tqdm
from datetime import datetime
import gc

# ==================== CONFIGURATION ====================
print("="*60)
print("MULTI-MODEL ENSEMBLE FOR PROTEIN FUNCTION PREDICTION")
print("="*60)

# ============ TUNING SECTION - EXPERIMENT WITH THESE ============

# Strategy 1: Model weights (MUST SUM TO 1.0)
WEIGHT_MODEL_A = 0.60  
WEIGHT_MODEL_B = 0.20  
WEIGHT_MODEL_C = 0.20 

# Validate weights
assert abs(WEIGHT_MODEL_A + WEIGHT_MODEL_B + WEIGHT_MODEL_C - 1.0) < 0.001, "Weights must sum to 1.0!"

# Strategy 2: Diversity bonus 
DIVERSITY_BONUS_MODE = 'adaptive'  # 'fixed', 'adaptive', 'multiplicative', or 'none'
DIVERSITY_BONUS_FIXED = 0.05
DIVERSITY_BONUS_MIN = 0.02
DIVERSITY_BONUS_MAX = 0.08  
MIN_SCORE_FOR_BONUS = 0.01  # Match với CONFIDENCE_THRESHOLD để không bỏ sót

# Strategy 3: Score calibration
APPLY_SCORE_CALIBRATION = False
CALIBRATION_POWER_A = 1.0
CALIBRATION_POWER_B = 1.0
CALIBRATION_POWER_C = 1.0

# Strategy 4: Filtering thresholds
CONFIDENCE_THRESHOLD = 0.01  # Giữ thấp cho Fmax
TOP_K_PER_PROTEIN = 1500

# Strategy 5: Conflict resolution
CONFLICT_RESOLUTION = 'weighted'  # 'weighted', 'max', 'min'

# ================================================================

# Memory optimization (KHÔNG THAY ĐỔI)
BATCH_SIZE = 1_000_000
EARLY_FILTER = True  # BẬT early filter để giảm RAM
MIN_SCORE_TO_KEEP = 0.01  # Lọc sớm predictions quá thấp
SCORE_CLIP_MIN = 0.0
SCORE_CLIP_MAX = 1.0

print(f"\n⚙️ Configuration:")
print(f"  - Model A weight: {WEIGHT_MODEL_A:.2f}")
print(f"  - Model B weight: {WEIGHT_MODEL_B:.2f}")
print(f"  - Model C weight: {WEIGHT_MODEL_C:.2f}")
print(f"  - Diversity bonus: {DIVERSITY_BONUS_MODE}")
print(f"  - Min score for bonus: {MIN_SCORE_FOR_BONUS}")
print(f"  - Score calibration: {APPLY_SCORE_CALIBRATION}")
print(f"  - Confidence threshold: {CONFIDENCE_THRESHOLD}")
print(f"  - Top-K per protein: {TOP_K_PER_PROTEIN}")

# ==================== HELPER FUNCTIONS ====================

def load_model_with_validation(filepath, model_name, calibration_power=1.0):
    """Load và validate model predictions với memory optimization"""
    start_time = time.time()
    print(f"\n📂 Loading {model_name}...")
    
    chunks = []
    chunk_size = 5_000_000
    
    for chunk in pd.read_csv(filepath, sep='\t', header=None, 
                              names=['protein', 'go_term', 'score'],
                              chunksize=chunk_size):
        # Clip scores
        chunk['score'] = chunk['score'].clip(SCORE_CLIP_MIN, SCORE_CLIP_MAX)
        
        # Score calibration
        if APPLY_SCORE_CALIBRATION and calibration_power != 1.0:
            chunk['score'] = np.power(chunk['score'], calibration_power)
        
        # Filter early nếu bật
        if EARLY_FILTER:
            chunk = chunk[chunk['score'] >= MIN_SCORE_TO_KEEP]
        
        chunk['key'] = chunk['protein'] + '|' + chunk['go_term']
        chunks.append(chunk)
    
    df = pd.concat(chunks, ignore_index=True)
    del chunks
    gc.collect()
    
    print(f"  ✓ Loaded {len(df):,} predictions")
    print(f"  ✓ Unique proteins: {df['protein'].nunique():,}")
    print(f"  ✓ Unique GO terms: {df['go_term'].nunique():,}")
    print(f"  ✓ Score range: [{df['score'].min():.4f}, {df['score'].max():.4f}]")
    print(f"  ✓ Mean score: {df['score'].mean():.4f}")
    
    # Remove duplicates
    duplicates = df['key'].duplicated().sum()
    if duplicates > 0:
        print(f"  ⚠️ Removing {duplicates:,} duplicates...")
        df = df.sort_values('score', ascending=False).drop_duplicates('key', keep='first')
        gc.collect()
    
    elapsed = time.time() - start_time
    print(f"  ⏱️ Loading time: {elapsed:.2f}s")
    print(f"  💾 Memory usage: ~{df.memory_usage(deep=True).sum() / 1e9:.2f} GB")
    
    return df

def calculate_adaptive_bonus(scores_list):
    """Bonus cao khi tất cả model TỰ TIN và ĐỒNG Ý (mở rộng cho 3 models)"""
    scores_array = np.array(scores_list).T  # shape: (n_predictions, n_models)
    
    # Tính agreement: 1 - std của scores
    agreement = 1 - np.std(scores_array, axis=1) / (np.mean(scores_array, axis=1) + 1e-8)
    agreement = np.clip(agreement, 0, 1)
    
    # Tính average confidence
    avg_confidence = np.mean(scores_array, axis=1)
    
    # Penalty nếu có model nào quá thấp
    min_score = np.min(scores_array, axis=1)
    confidence_factor = min_score / MIN_SCORE_FOR_BONUS if MIN_SCORE_FOR_BONUS > 0 else 1.0
    confidence_factor = np.clip(confidence_factor, 0, 1)
    
    bonus = (DIVERSITY_BONUS_MIN + 
             (DIVERSITY_BONUS_MAX - DIVERSITY_BONUS_MIN) * agreement * avg_confidence)
    
    return bonus * confidence_factor

def calculate_multiplicative_bonus(scores_list, weights):
    """Multiplicative ensemble - tốt khi tất cả model tự tin (mở rộng cho 3 models)"""
    scores_array = np.array(scores_list).T  # shape: (n_predictions, n_models)
    
    # Geometric mean (nth root of product)
    geometric_mean = np.prod(scores_array, axis=1) ** (1.0 / len(scores_list))
    
    # Weighted average
    weighted_avg = np.dot(scores_array, weights)
    
    # Blend giữa weighted average và geometric mean
    alpha = 0.2  # 20% geometric, 80% weighted
    return alpha * geometric_mean + (1 - alpha) * weighted_avg

# ==================== MAIN PIPELINE ====================

overall_start = time.time()

# Load all models
model_a = load_model_with_validation(
    '/kaggle/input/prediction-with-goa-uniprot-all/submission.tsv',
    'Model A (GOA-UniProt)',
    calibration_power=CALIBRATION_POWER_A
)

model_b = load_model_with_validation(
    '/kaggle/input/cafa-6-protein-function-starter-eda-model/submission.tsv',
    'Model B (Starter EDA)',
    calibration_power=CALIBRATION_POWER_B
)

model_c = load_model_with_validation(
    '/kaggle/input/k/daovanda2405/cafa-6-protein-function-starter-eda-model/submission.tsv',
    'Model C (Custom Model)',
    calibration_power=CALIBRATION_POWER_C
)

# Analyze overlap
print(f"\n🔨 Analyzing overlap...")
keys_a = set(model_a['key'])
keys_b = set(model_b['key'])
keys_c = set(model_c['key'])

all_keys = keys_a | keys_b | keys_c
overlap_ab = keys_a & keys_b
overlap_ac = keys_a & keys_c
overlap_bc = keys_b & keys_c
overlap_abc = keys_a & keys_b & keys_c

only_a = len(keys_a - keys_b - keys_c)
only_b = len(keys_b - keys_a - keys_c)
only_c = len(keys_c - keys_a - keys_b)

print(f"  ✓ Total unique predictions: {len(all_keys):,}")
print(f"  ✓ Only in Model A: {only_a:,} ({only_a/len(all_keys)*100:.1f}%)")
print(f"  ✓ Only in Model B: {only_b:,} ({only_b/len(all_keys)*100:.1f}%)")
print(f"  ✓ Only in Model C: {only_c:,} ({only_c/len(all_keys)*100:.1f}%)")
print(f"  ✓ Overlap A∩B: {len(overlap_ab):,} ({len(overlap_ab)/len(all_keys)*100:.1f}%)")
print(f"  ✓ Overlap A∩C: {len(overlap_ac):,} ({len(overlap_ac)/len(all_keys)*100:.1f}%)")
print(f"  ✓ Overlap B∩C: {len(overlap_bc):,} ({len(overlap_bc)/len(all_keys)*100:.1f}%)")
print(f"  ✓ Overlap A∩B∩C: {len(overlap_abc):,} ({len(overlap_abc)/len(all_keys)*100:.1f}%)")

# Calculate overlap score statistics
if len(overlap_abc) > 0:
    overlap_a_scores = model_a[model_a['key'].isin(overlap_abc)]['score']
    overlap_b_scores = model_b[model_b['key'].isin(overlap_abc)]['score']
    overlap_c_scores = model_c[model_c['key'].isin(overlap_abc)]['score']
    print(f"  ✓ Overlap (A∩B∩C) avg score A: {overlap_a_scores.mean():.4f}")
    print(f"  ✓ Overlap (A∩B∩C) avg score B: {overlap_b_scores.mean():.4f}")
    print(f"  ✓ Overlap (A∩B∩C) avg score C: {overlap_c_scores.mean():.4f}")

del keys_a, keys_b, keys_c, all_keys, overlap_ab, overlap_ac, overlap_bc, overlap_abc
gc.collect()

# Merge strategy
print(f"\n🚀 Creating ensemble (memory-efficient method)...")
start_time = time.time()

model_a_clean = model_a[['key', 'protein', 'go_term', 'score']].rename(columns={'score': 'score_a'})
model_b_clean = model_b[['key', 'score']].rename(columns={'score': 'score_b'})
model_c_clean = model_c[['key', 'score']].rename(columns={'score': 'score_c'})

del model_a, model_b, model_c
gc.collect()

print(f"  📊 Merging models...")
# Merge A and B first
ensemble = model_a_clean.merge(model_b_clean, on='key', how='outer')
del model_b_clean
gc.collect()

# Then merge with C
ensemble = ensemble.merge(model_c_clean, on='key', how='outer')
del model_a_clean, model_c_clean
gc.collect()

# Fill NaN
ensemble['score_a'] = ensemble['score_a'].fillna(0)
ensemble['score_b'] = ensemble['score_b'].fillna(0)
ensemble['score_c'] = ensemble['score_c'].fillna(0)

# Free memory immediately
gc.collect()

print(f"  ✓ Merged: {len(ensemble):,} total predictions")
print(f"  💾 Memory usage: ~{ensemble.memory_usage(deep=True).sum() / 1e9:.2f} GB")

# FILTER EARLY để giảm data trước khi tính toán phức tạp
print(f"  🔍 Early filtering to reduce memory...")
initial_count = len(ensemble)

# Chỉ giữ predictions có ít nhất 1 model score > threshold
mask_keep = (ensemble['score_a'] >= MIN_SCORE_TO_KEEP) | \
            (ensemble['score_b'] >= MIN_SCORE_TO_KEEP) | \
            (ensemble['score_c'] >= MIN_SCORE_TO_KEEP)

ensemble = ensemble[mask_keep].copy()
del mask_keep
gc.collect()

filtered_count = len(ensemble)
print(f"    ✓ Kept {filtered_count:,}/{initial_count:,} predictions ({filtered_count/initial_count*100:.1f}%)")
print(f"    ✓ Freed ~{(initial_count - filtered_count) * 200 / 1e9:.2f} GB")
print(f"    💾 Current memory: ~{ensemble.memory_usage(deep=True).sum() / 1e9:.2f} GB")

# Calculate base ensemble score
print(f"  🧮 Calculating ensemble scores (mode: {DIVERSITY_BONUS_MODE})...")

if DIVERSITY_BONUS_MODE == 'multiplicative':
    # Multiplicative ensemble
    mask_all = (ensemble['score_a'] > 0) & (ensemble['score_b'] > 0) & (ensemble['score_c'] > 0)
    
    # Base weighted average
    ensemble['score'] = (ensemble['score_a'] * WEIGHT_MODEL_A + 
                        ensemble['score_b'] * WEIGHT_MODEL_B + 
                        ensemble['score_c'] * WEIGHT_MODEL_C)
    
    if mask_all.sum() > 0:
        weights = np.array([WEIGHT_MODEL_A, WEIGHT_MODEL_B, WEIGHT_MODEL_C])
        scores_list = [
            ensemble.loc[mask_all, 'score_a'].values,
            ensemble.loc[mask_all, 'score_b'].values,
            ensemble.loc[mask_all, 'score_c'].values
        ]
        ensemble.loc[mask_all, 'score'] = calculate_multiplicative_bonus(scores_list, weights)
        print(f"    ✓ Multiplicative bonus applied to {mask_all.sum():,} predictions")
    
    ensemble['bonus'] = 0.0
    
else:
    # Weighted average base
    ensemble['weighted_score'] = (
        ensemble['score_a'] * WEIGHT_MODEL_A + 
        ensemble['score_b'] * WEIGHT_MODEL_B +
        ensemble['score_c'] * WEIGHT_MODEL_C
    )
    
    # Apply diversity bonus
    print(f"  🎁 Applying diversity bonus...")
    if DIVERSITY_BONUS_MODE == 'adaptive':
        # Apply bonus cho tất cả predictions có cả 3 models predict (không cần threshold cao)
        mask_all = ((ensemble['score_a'] >= MIN_SCORE_FOR_BONUS) & 
                    (ensemble['score_b'] >= MIN_SCORE_FOR_BONUS) & 
                    (ensemble['score_c'] >= MIN_SCORE_FOR_BONUS))
        
        ensemble['bonus'] = 0.0
        n_bonus = mask_all.sum()
        
        if n_bonus > 0:
            print(f"    ℹ️ Computing bonus for {n_bonus:,} predictions...")
            
            # Xử lý theo BATCH để tránh tràn RAM
            batch_size = 5_000_000
            if n_bonus > batch_size:
                print(f"    ℹ️ Processing in batches of {batch_size:,}...")
                indices = ensemble.index[mask_all].tolist()
                
                for i in range(0, len(indices), batch_size):
                    batch_idx = indices[i:i+batch_size]
                    scores_list = [
                        ensemble.loc[batch_idx, 'score_a'].values,
                        ensemble.loc[batch_idx, 'score_b'].values,
                        ensemble.loc[batch_idx, 'score_c'].values
                    ]
                    ensemble.loc[batch_idx, 'bonus'] = calculate_adaptive_bonus(scores_list)
                    
                    if i % (batch_size * 5) == 0:
                        print(f"      Processed {i:,}/{n_bonus:,} predictions...")
                    
                    del scores_list, batch_idx
                    gc.collect()
            else:
                scores_list = [
                    ensemble.loc[mask_all, 'score_a'].values,
                    ensemble.loc[mask_all, 'score_b'].values,
                    ensemble.loc[mask_all, 'score_c'].values
                ]
                ensemble.loc[mask_all, 'bonus'] = calculate_adaptive_bonus(scores_list)
                del scores_list
                gc.collect()
            
            print(f"    ✓ Bonus applied to {n_bonus:,} predictions ({n_bonus/len(ensemble)*100:.1f}%)")
            del mask_all
            gc.collect()
        else:
            print(f"    ⚠️ No predictions meet bonus criteria (MIN_SCORE_FOR_BONUS={MIN_SCORE_FOR_BONUS})")
    
    elif DIVERSITY_BONUS_MODE == 'fixed':
        mask_all = (ensemble['score_a'] > 0) & (ensemble['score_b'] > 0) & (ensemble['score_c'] > 0)
        ensemble['bonus'] = 0.0
        n_bonus = mask_all.sum()
        ensemble.loc[mask_all, 'bonus'] = DIVERSITY_BONUS_FIXED
        print(f"    ✓ Fixed bonus applied to {n_bonus:,} predictions ({n_bonus/len(ensemble)*100:.1f}%)")
        del mask_all
        gc.collect()
    
    else:
        ensemble['bonus'] = 0.0
    
    # Final score
    ensemble['score'] = (ensemble['weighted_score'] + ensemble['bonus']).clip(upper=SCORE_CLIP_MAX)

# Conflict resolution
if CONFLICT_RESOLUTION == 'max':
    # Take max of individual scores or ensemble
    ensemble['score'] = ensemble[['score_a', 'score_b', 'score_c', 'score']].max(axis=1)
elif CONFLICT_RESOLUTION == 'min':
    # Conservative: take min where all predict
    mask_all = (ensemble['score_a'] > 0) & (ensemble['score_b'] > 0) & (ensemble['score_c'] > 0)
    ensemble.loc[mask_all, 'score'] = ensemble.loc[mask_all, ['score_a', 'score_b', 'score_c']].min(axis=1)

# Drop intermediate columns để giảm memory
ensemble = ensemble[['protein', 'go_term', 'score']].copy()
gc.collect()

print(f"  💾 Memory after cleanup: ~{ensemble.memory_usage(deep=True).sum() / 1e9:.2f} GB")

elapsed = time.time() - start_time
print(f"  ✓ Ensemble created in {elapsed:.2f}s")

# Filtering
print(f"\n🔍 Filtering predictions...")
print(f"  - Before final filter: {len(ensemble):,} predictions")

# Apply confidence threshold
if CONFIDENCE_THRESHOLD > 0:
    before = len(ensemble)
    ensemble = ensemble[ensemble['score'] >= CONFIDENCE_THRESHOLD].copy()
    gc.collect()
    after = len(ensemble)
    print(f"  - After confidence filter (≥{CONFIDENCE_THRESHOLD}): {after:,} predictions (removed {before-after:,})")
else:
    print(f"  - No confidence threshold applied")

# Top-K per protein
if TOP_K_PER_PROTEIN is not None:
    print(f"  - Applying top-{TOP_K_PER_PROTEIN} per protein filter...")
    ensemble = ensemble.sort_values('score', ascending=False)
    ensemble = ensemble.groupby('protein').head(TOP_K_PER_PROTEIN).reset_index(drop=True)
    gc.collect()
    print(f"  - After top-K filter: {len(ensemble):,} predictions")

# Sort by score
ensemble = ensemble.sort_values('score', ascending=False)

# Statistics
print(f"\n📊 Final Statistics:")
print(f"  - Total predictions: {len(ensemble):,}")
print(f"  - Proteins covered: {ensemble['protein'].nunique():,}")
print(f"  - GO terms covered: {ensemble['go_term'].nunique():,}")
print(f"  - Avg predictions per protein: {len(ensemble)/ensemble['protein'].nunique():.1f}")
print(f"  - Score distribution:")
print(f"    • Mean: {ensemble['score'].mean():.4f}")
print(f"    • Median: {ensemble['score'].median():.4f}")
print(f"    • Std: {ensemble['score'].std():.4f}")
print(f"    • Min: {ensemble['score'].min():.4f}")
print(f"    • Max: {ensemble['score'].max():.4f}")

# Save submission
print(f"\n💾 Saving submission...")
output_file = 'submission.tsv'
ensemble[['protein', 'go_term', 'score']].to_csv(
    output_file, 
    sep='\t', 
    index=False, 
    header=False
)
print(f"  ✓ Saved to: {output_file}")

# Total time
total_time = time.time() - overall_start
print(f"\n⏱️ Total execution time: {total_time:.2f}s ({total_time/60:.1f} minutes)")
print("="*60)
print("✅ ENSEMBLE COMPLETE!")
print(f"🎯 Experiment with different parameters to optimize score!")
print("="*60)

MULTI-MODEL ENSEMBLE FOR PROTEIN FUNCTION PREDICTION

⚙️ Configuration:
  - Model A weight: 0.60
  - Model B weight: 0.20
  - Model C weight: 0.20
  - Diversity bonus: adaptive
  - Min score for bonus: 0.01
  - Score calibration: False
  - Confidence threshold: 0.01
  - Top-K per protein: 1500

📂 Loading Model A (GOA-UniProt)...
  ✓ Loaded 51,391,680 predictions
  ✓ Unique proteins: 279,437
  ✓ Unique GO terms: 32,618
  ✓ Score range: [0.0100, 1.0000]
  ✓ Mean score: 0.2410
  ⚠️ Removing 2,513,060 duplicates...
  ⏱️ Loading time: 201.72s
  💾 Memory usage: ~10.76 GB

📂 Loading Model B (Starter EDA)...
  ✓ Loaded 14,037,195 predictions
  ✓ Unique proteins: 202,893
  ✓ Unique GO terms: 31,451
  ✓ Score range: [0.0102, 1.0000]
  ✓ Mean score: 0.2772
  ⏱️ Loading time: 30.52s
  💾 Memory usage: ~2.98 GB

📂 Loading Model C (Custom Model)...
  ✓ Loaded 46,564,288 predictions
  ✓ Unique proteins: 141,864
  ✓ Unique GO terms: 2,200
  ✓ Score range: [0.0100, 0.9990]
  ✓ Mean score: 0.0528
  ⏱️ Lo

/usr/local/lib/python3.11/dist-packages/pandas/core/groupby/indexing.py:223: RuntimeWarning: invalid value encountered in less
  mask &= self._ascending_count < stop


  - After top-K filter: 46,190,852 predictions

📊 Final Statistics:
  - Total predictions: 46,190,852
  - Proteins covered: 279,437
  - GO terms covered: 32,618
  - Avg predictions per protein: 165.3
  - Score distribution:
    • Mean: 0.1557
    • Median: 0.0552
    • Std: 0.2118
    • Min: 0.0100
    • Max: 1.0000

💾 Saving submission...
  ✓ Saved to: submission.tsv

⏱️ Total execution time: 1789.80s (29.8 minutes)
✅ ENSEMBLE COMPLETE!
🎯 Experiment with different parameters to optimize score!
